In [111]:
import numpy as np
import pandas as pd
import pickle as pi

from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler

#### Объединение датафреймов

In [112]:
# загружаем датасеты дескрипторов
df_mordred = pd.read_csv('db_descriptors_Mordred.csv')
df_rdkit = pd.read_csv('db_descriptors_RDKit.csv')
df_pubchem = pd.read_csv('db_descriptors_PubChem.csv')
df_matproj = pd.read_csv('db_descriptors_MaterialsProject.csv')

C:\Users\ninag\AppData\Local\Temp\ipykernel_16260\2548817588.py:2: DtypeWarning: Columns (3,8,9,10,11,12,13,14,15,16,17,18,19,57,58,59,60,61,62,63,64,65,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,139,140,141,142,143,144,145,146,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,166,167,168,169,170,171,172,173,175,176,177,178,179,180,181,182,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,220,221,222,223,224,225,226,227,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,264,265,266,267,268,269,270,271,272,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,345,346,347,348,349,350,351,352,353,355,356,357,358,359,360,361,362,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,382,383,384,385,386,387,388,389,391,392,393,394,395,396,397,39

In [113]:
# заменяем все строковые значения на nans и убеждаемся, что все элементы имеют числовой формат
for df in [df_mordred, df_rdkit, df_pubchem, df_matproj]:
    columns = df.columns.tolist()

    for column in columns[4:]:
        new_column = pd.to_numeric(df[column], errors='coerce')
        df[column] = new_column

In [114]:
# отсортируем значения внутри датафреймов и убедимся, что строки находятся на одинаковых местах
df_mordred_sorted = df_mordred.sort_values(by=['raw_value', 'normalised_name']).reset_index(drop=True)
df_rdkit_sorted = df_rdkit.sort_values(by=['raw_value', 'normalised_name']).reset_index(drop=True)
df_pubchem_sorted = df_pubchem.sort_values(by=['raw_value', 'normalised_name']).reset_index(drop=True)
df_matproj_sorted = df_matproj.sort_values(by=['raw_value', 'normalised_name']).reset_index(drop=True)

test = []
for df_1 in [df_mordred_sorted, df_rdkit_sorted, df_pubchem_sorted, df_matproj_sorted]:
    for df_2 in [df_mordred_sorted, df_rdkit_sorted, df_pubchem_sorted, df_matproj_sorted]:
        test.append(df_1['normalised_name'].equals(df_2['normalised_name']))

if False not in test:
    print('Датафреймы отсортированы одинаково!')

Датафреймы отсортированы одинаково!


In [115]:
# соединим датафреймы
df = pd.concat([df_mordred_sorted, df_rdkit_sorted.iloc[:, 4:], df_pubchem_sorted.iloc[:, 10:], df_matproj_sorted.iloc[:, 4:]], axis=1)

In [116]:
# удалим оставшиеся ненужные колонки
df = df.drop(['Name', 'normalised_name', 'mol_file'], axis=1)
df.shape

(1334, 1901)

#### Удаление пустых колонок и строк

In [117]:
# удалим колонки и строки, состоявшие полностью из NaN (thresh = 2 - заполнено что-то кроме raw vallue)
df = df.dropna(axis = 1, how='all')
df = df.dropna(axis = 0, thresh=2)
df.shape

(1071, 1809)

In [118]:
# удалим дублирующиеся по содержанию колонки
df_no_duplicates = df.T.drop_duplicates().T
df_no_duplicates = df_no_duplicates.iloc[:, ~ df_no_duplicates.columns.duplicated()]
df_no_duplicates.shape

(1071, 1579)

In [119]:
# найдем колонки, в которых nan больше 30% и удалим их из датафрейма
drop_list = []

for column in df_no_duplicates.columns:
    count_nan = df_no_duplicates[column].isnull().sum().astype(int)

    if count_nan >= len(df_no_duplicates)*0.30:
        drop_list.append(column)

print('Число колонок, которые необходимо удалить:', len(drop_list))

Число колонок, которые необходимо удалить: 461


In [120]:
# отфильтруем низкодисперсные дескрипторы
variances = []

for col in df_no_duplicates.columns:
    var = df_no_duplicates[col].var(ddof=1)
    variances.append((col, var))

for col, var in variances:
  if var <= 0.01:
    drop_list.append(col)

df_result = df_no_duplicates.drop(drop_list, axis=1)

print('Итоговый размер датафрейма:', df_result.shape)

Итоговый размер датафрейма: (1071, 1034)


#### Заполнение пустых ячеек

In [121]:
# зададим часть датафрейма, в которой необходимо заполнить nans
X = df_result.iloc[:, 1:]

In [122]:
# из-за возникающей ошибки с наличием бесконечных значений применим функцю
X.replace([np.inf, -np.inf], np.nan, inplace=True)

C:\Users\ninag\AppData\Local\Temp\ipykernel_16260\3711392247.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.replace([np.inf, -np.inf], np.nan, inplace=True)


In [123]:
# количество nans
pd.DataFrame(X).isnull().sum().sum()

209013

In [124]:
# define imputer
imputer = KNNImputer()

# fit on the dataset
imputer.fit(X)

# transform the dataset
Xtrans = imputer.transform(X)

c:\Users\ninag\mambaforge\envs\math\lib\site-packages\sklearn\metrics\pairwise.py:478: RuntimeWarning: overflow encountered in multiply
  YY = Y * Y
c:\Users\ninag\mambaforge\envs\math\lib\site-packages\sklearn\metrics\pairwise.py:480: RuntimeWarning: invalid value encountered in subtract
  distances -= np.dot(missing_X, YY.T)


In [125]:
# переведем в датафрейм
X_result = pd.DataFrame(Xtrans, columns=X.columns)

In [126]:
# количество nans после заполнения
X_result.isnull().sum().sum()

0

In [127]:
# сохраним полученные данные
X_result.insert(0, "raw_value", df_result["raw_value"].reset_index(drop=True), True)
X_result.to_csv('6_clear.csv', index=False)

In [135]:
X_result = X_result.drop(['raw_value'], axis=1)

#### Нормализация

In [136]:
# зададим scaler
scaler = MinMaxScaler()

In [137]:
# затем применим его к набору данных
scaler.fit(X_result)

MinMaxScaler()

In [138]:
# сохраним scaler
pi.dump(scaler, open('scaler.pkl', 'wb'))

In [139]:
# трансформируем имеющиеся данные
X_result_transform = scaler.transform(X_result)

In [140]:
# вернем в датафрейм raw value
X_result_transform = pd.DataFrame(X_result_transform, columns=X.columns)
X_result_transform.insert(0, "raw_value", df_result["raw_value"].reset_index(drop=True), True)

In [141]:
# сохраним полученные нормализованные данные
X_result_transform.to_csv('6_clear_normalised.csv', index=False)